In [1]:
import os
import numpy as np
import cv2

def read_images(path, image_size):
    names = []
    training_images, training_labels = [], []
    label = 0
    for dirname, subdirnames, filename in os.walk(path):
        for subdirname in subdirnames:
            names.append(subdirname)
            subject_path = os.path.join(dirname, subdirname)
            for filename in os.listdir(subject_path):
                img = cv2.imread(os.path.join(subject_path, filename),cv2.IMREAD_GRAYSCALE)
                if img is None:
                    #The file cannot be loaded as an image. Skip it.
                    continue
                img = cv2.resize(img, image_size)
                training_images.append(img)
                training_labels.append(label)
            label += 1
    training_images = np.asarray(training_images, np.uint8)
    training_labels = np.asarray(training_labels, np.int32)
    return names, training_images, training_labels

path_to_training_images = '../data/at'#Path where training images are stored. The training images can be generated by the
                                      # script 'GenerateFaceRecognitionData.ipynb'
training_image_size = (200, 200)
names, training_images, training_labels = read_images(path_to_training_images, training_image_size)

#-------------------Set up and train the model-------------------------------------------------------------------------------
model = cv2.face.EigenFaceRecognizer_create()
model.train(training_images, training_labels)

face_cascade = cv2.CascadeClassifier('./cascades/haarcascade_frontalface_default.xml')
#------------------Capture and recognize the face----------------------------------------------------------------------------
camera = cv2.VideoCapture(0)
while (cv2.waitKey(1) == -1):
    success, frame = camera.read()
    if success:
        faces = face_cascade.detectMultiScale(frame, 1.3, 5)
        for (x,y,w,h) in faces:
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255), 3)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            roi_gray = gray[x:x+w, y:y+h]
            if roi_gray.size == 0:
                continue
            roi_gray = cv2.resize(roi_gray, training_image_size)
            label, confidence = model.predict(roi_gray)
            text = '%s, confidence = %.2f' % (names[label], confidence)
            cv2.putText(frame, text, (x, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)
        cv2.imshow('Face Recognition', frame)
cv2.destroyAllWindows()